In [ ]:
pip install openai

In [ ]:
pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
openai_key = os.environ.get('OPENAI_API_KEY')

client = OpenAI(
    api_key = openai_key
)

In [ ]:
from dataclasses import dataclass

@dataclass
class ExternalKnowledge:
    title: str
    content: str
    
    def __str__(self) -> str:
        return f"{self.title}: {self.content}"

wafa = """
    Muhammad Wafa Al Ausath, or commonly known as Wafa, is a student majoring in Informatics Engineering at Lampung University.
    Wafa has a strong interest in the field of information technology, especially mobile application development.
    He has experience as a mobile application developer at Coolva Technology and has worked as a freelance mobile developer.
    He has also served as a mobile developer intern at several startup companies.
    
    github profile = https://github.com/mikirinkode
    linkedin profile = www.linkedin.com/in/muhammadwafaalausath
    personal website = https://mikirinkode.com
""".strip()

extra_knowledges = [
    ExternalKnowledge(title="Wafa",content=wafa),
]

In [119]:
import json
import datetime

class MyAssistant:
    def __init__(self, openai_client, extra_knowledges, name = "Miriko"):
        self.name = name
        self.messages = [
            {"role": "system", "content": f"""
             You are a Wafa's personal AI Assistant that helps him with his daily tasks. 
             Here some extra knowledges for you {extra_knowledges}.
             """.strip()},
        ]
        self.client = openai_client
        print(self.greet())

    def greet(self):
        # print(self.chat("Say Greeting"))
        return f"Hello, {self.name} here. Anything i could help today?"

    # promt -> input user
    # system_content -> role the ai that we want
    # model -> used gpt model
    # stream -> to display the result in real time (like typing)
    def chat(self, prompt, system_content = None, model = "gpt-3.5-turbo", stream = False):
        
        system_content = system_content if system_content != None else self._determine_system_content(prompt)
            
        self.messages.append({"role": "system", "content": system_content}) 
        
        self.messages.append({"role": "user", "content": prompt})
        
        if stream:
            completion = self.client.chat.completions.create(
                model = model,
                messages = self.messages,
                stream = True
            )
            for chunk in completion:
                if chunk.choices[0].delta.content is not None:
                    print(chunk.choices[0].delta.content, end="")
        else: 
            completion = client.chat.completions.create(
                model = model,
                messages = self.messages
            )
            result = completion.choices[0].message.content
            self._handle_result(result)
            
            
    def _determine_system_content(self, content):
        if "summarize" in content:
            return self._summarizer()
        elif "translate" in content:
            return self._translator()
        elif "correct" in content:
            return self._english_grammar_master()
        elif "who is" in content:
            return self._who_is()
        elif "how to" in content:
            return self._how_to()
        else:
            return """
        You are a Wafa's personal AI Assistant that helps him with his daily tasks.
             
             You also have this following tools:
             Time () -> return current time
             Date () -> return current date
             
             Output in the following format:
             {
                 direct_answer: "<your answer", # if you have direct answer
                 invoke_external_tool: "<tool name>([parameter])", # if you want to invoke external tool
             }
        """
            
    def _handle_result(self, result):
        json_result = json.loads(result)
        
        if "direct_answer" in result:
            direct_answer = json_result.get("direct_answer")
            print(direct_answer)
        elif "invoke_external_tool" in result:
            tool_name = json_result.get("invoke_external_tool")
            if tool_name == "Time()":
                print(datetime.datetime.now().strftime("%H:%M"))
            elif tool_name == "Date()":
                print(datetime.datetime.now().strftime("%A, %d %B %Y"))
        else:
            print("Here is the result:")
            print(result)
    
    
    def _summarizer(self):
        return """
                        You are a summarizer AI that summarizes the given content in two sentences.
                        The output format should be in a list
                    """.strip()

    def _translator(self):
        return """
                        You are a translator AI that translates the given content to user's requested language.
                        Don't make assumptions about what language the user wants to translate to.
                        Ask for clarification if a user doesn't specify a language.
                    """.strip()
        
    def _english_grammar_master(self):
        return """You are an English grammar master that corrects the given content.
                    """.strip()

    def _who_is(self):
        return """if the user asks about a person, return in this JSON format:
                            {
                                "name": "John Doe", # must be a full name
                                "skills": ["Mobile Development", "Web Development", "Machine Learning"],
                                "experience": [ # The experience must be clear information, if there is some missing information use empty string
                                    {
                                        "company": "Coolva Technology",
                                        "role": "Mobile Developer",
                                        "duration": 1 # must be return in year
                                    },
                                    {
                                        "company": "Gojek",
                                        "role": "Software Engineer",
                                        "duration": 1 # must be return in year
                                    }
                                ]
                            }
                            
                            if the user ask about How To Do something, 
                         """.strip()
        
    def _how_to(self):
        return """please return in this JSON List format:
                            [
                                {
                                    "step" : "step name",
                                    "description": "step description",
                                    "tools" : ["tool 1", "tool 2"], # optional, if there is a tool that needed
                                    "difficulty": "" # can be easy, medium, hard, or empty string
                                }
                            ]
                    """.strip()
        
    def display_chat_history(self):
        for message in self.messages:
            print(message["role"], ":", message["content"])

In [120]:
my_assistant = MyAssistant(openai_client=client, extra_knowledges=extra_knowledges)

Hello, Miriko here. Anything i could help today?


In [ ]:
my_assistant.chat("do you know who am i?")

In [93]:
my_assistant.chat("who is wafa?")

Wafa, whose full name is Muhammad Wafa Al Ausath, is a student majoring in Informatics Engineering at Lampung University. He has a strong interest in the field of information technology, particularly in mobile application development. Wafa has experience working as a mobile application developer at Coolva Technology and has also worked as a freelance mobile developer. Additionally, he has interned as a mobile developer at several startup companies. You can find more information about him on his personal website at https://mikirinkode.com and on his GitHub profile at https://github.com/mikirinkode.


In [121]:
my_assistant.chat("how to make an instant noodle?")

Sorry, I don't understand
result:  [
    {
        "step": "Boil water",
        "description": "Fill a pot with water and put it on the stove. Turn on the heat and bring the water to a boil.",
        "tools": ["pot", "stove"],
        "difficulty": "easy"
    },
    {
        "step": "Open the noodle package",
        "description": "Open the package of instant noodles and take out the noodles and seasoning packets.",
        "difficulty": "easy"
    },
    {
        "step": "Put noodles in boiling water",
        "description": "Once the water is boiling, add the noodles to the pot. Let them cook for a few minutes until they become soft.",
        "tools": ["pot"],
        "difficulty": "easy"
    },
    {
        "step": "Add seasoning",
        "description": "Open the seasoning packet and pour it into the pot with the cooked noodles. Stir well to distribute the seasoning evenly.",
        "tools": ["spoon"],
        "difficulty": "easy"
    },
    {
        "step": "Serve and enj

In [ ]:
my_assistant.chat("i want 'hello, how are you?' be translated in another language")

In [ ]:
my_assistant.chat("in japanese please")

In [112]:
my_assistant.display_chat_history()

system : You are a Wafa's personal AI Assistant that helps him with his daily tasks. 
             Here some extra knowledges for you [ExternalKnowledge(title='Wafa', content='Muhammad Wafa Al Ausath, or commonly known as Wafa, is a student majoring in Informatics Engineering at Lampung University.\n    Wafa has a strong interest in the field of information technology, especially mobile application development.\n    He has experience as a mobile application developer at Coolva Technology and has worked as a freelance mobile developer.\n    He has also served as a mobile developer intern at several startup companies.\n    \n    github profile = https://github.com/mikirinkode\n    linkedin profile = www.linkedin.com/in/muhammadwafaalausath\n    personal website = https://mikirinkode.com')].
system : 
        You are a Wafa's personal AI Assistant that helps him with his daily tasks.
             
             You also have this following tools:
             Time () -> return current time

In [111]:
my_assistant.chat("what time is it now?")

16:46:33


In [117]:
my_assistant.chat("what date is it now?")

Saturday, 30 December 2023
